In [ ]:
# for each value in Box_Score_Link, add "https://www.basketball-reference.com/"
# get a fucntion that pulls in table class="sortable stats_table now_sortable" and 'id=box-{PHI}-game-basic'
# and table class="sortable stats_table now_sortable"
# concat the two

# then the same for other team

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# HTML content
url = 'https://www.basketball-reference.com/leagues/NBA_2023_games.html'
response = requests.get(url)

# Create a BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table element
table = soup.find('table', class_='suppress_glossary sortable stats_table')

# Find all the rows in the table body
rows = table.tbody.find_all('tr')

# Create empty lists to store the extracted data
dates = []
visitor_neutrals = []
visitor_pts_list = []
home_neutrals = []
home_pts_list = []
box_score_links = []
arenas = []

# Iterate over each row and extract the desired information
for row in rows:
    date = row.find('th', {'data-stat': 'date_game'}).text.strip()
    visitor_neutral = row.find('td', {'data-stat': 'visitor_team_name'}).text.strip()
    visitor_pts = row.find('td', {'data-stat': 'visitor_pts'}).text.strip()
    home_neutral = row.find('td', {'data-stat': 'home_team_name'}).text.strip()
    home_pts = row.find('td', {'data-stat': 'home_pts'}).text.strip()
    box_score_link = 'https://www.basketball-reference.com' + row.find('td', {'data-stat': 'box_score_text'}).a['href']
    arena = row.find('td', {'data-stat': 'arena_name'}).text.strip()
    
    dates.append(date)
    visitor_neutrals.append(visitor_neutral)
    visitor_pts_list.append(visitor_pts)
    home_neutrals.append(home_neutral)
    home_pts_list.append(home_pts)
    box_score_links.append(box_score_link)
    arenas.append(arena)

# Create empty lists to store the scraped data
teams = []
stats_data = []

# Scrape data from each box score link
for link in box_score_links:
    box_score_response = requests.get(link)
    box_score_soup = BeautifulSoup(box_score_response.text, 'html.parser')
    
    # Find the table element with class "sortable stats_table" and an id containing "game-basic"
    stats_table = box_score_soup.find('table', class_='sortable stats_table', id=lambda x: x and 'game-basic' in x)
    
    if stats_table:
        # Extract the team abbreviation from the id attribute
        team_abbr = stats_table['id'].split('-')[1].upper()
        
        # Find all the rows in the table body
        stats_rows = stats_table.tbody.find_all('tr')
        
        # Extract the desired information from each row
        for row in stats_rows:
            player = row.find('th', {'data-stat': 'player'})
            player = player.text.strip() if player else ''
            
            mp = row.find('td', {'data-stat': 'mp'})
            mp = mp.text.strip() if mp else ''
            
            fg = row.find('td', {'data-stat': 'fg'})
            fg = fg.text.strip() if fg else ''
            
            fga = row.find('td', {'data-stat': 'fga'})
            fga = fga.text.strip() if fga else ''
            
            fg_pct = row.find('td', {'data-stat': 'fg_pct'})
            fg_pct = fg_pct.text.strip() if fg_pct else ''
            
            fg3 = row.find('td', {'data-stat': 'fg3'})
            fg3 = fg3.text.strip() if fg3 else ''
            
            fg3a = row.find('td', {'data-stat': 'fg3a'})
            fg3a = fg3a.text.strip() if fg3a else ''
            
            fg3_pct = row.find('td', {'data-stat': 'fg3_pct'})
            fg3_pct = fg3_pct.text.strip() if fg3_pct else ''
            
            ft = row.find('td', {'data-stat': 'ft'})
            ft = ft.text.strip() if ft else ''
            
            fta = row.find('td', {'data-stat': 'fta'})
            fta = fta.text.strip() if fta else ''
            
            ft_pct = row.find('td', {'data-stat': 'ft_pct'})
            ft_pct = ft_pct.text.strip() if ft_pct else ''
            
            orb = row.find('td', {'data-stat': 'orb'})
            orb = orb.text.strip() if orb else ''
            
            drb = row.find('td', {'data-stat': 'drb'})
            drb = drb.text.strip() if drb else ''
            
            trb = row.find('td', {'data-stat': 'trb'})
            trb = trb.text.strip() if trb else ''
            
            ast = row.find('td', {'data-stat': 'ast'})
            ast = ast.text.strip() if ast else ''
            
            stl = row.find('td', {'data-stat': 'stl'})
            stl = stl.text.strip() if stl else ''
            
            blk = row.find('td', {'data-stat': 'blk'})
            blk = blk.text.strip() if blk else ''
            
            tov = row.find('td', {'data-stat': 'tov'})
            tov = tov.text.strip() if tov else ''
            
            pf = row.find('td', {'data-stat': 'pf'})
            pf = pf.text.strip() if pf else ''
            
            pts = row.find('td', {'data-stat': 'pts'})
            pts = pts.text.strip() if pts else ''
            
            plus_minus = row.find('td', {'data-stat': 'plus_minus'})
            plus_minus = plus_minus.text.strip() if plus_minus else ''
            
            stats_data.append([team_abbr, player, mp, fg, fga, fg_pct, fg3, fg3a, fg3_pct, ft, fta, ft_pct, orb, drb, trb, ast, stl, blk, tov, pf, pts, plus_minus])
    else:
        print(f"No stats table found for {link}")

# Create a DataFrame using the extracted data
data = {
    'Date': dates,
    'Visitor/Neutral': visitor_neutrals,
    'Visitor Points': visitor_pts_list,
    'Home/Neutral': home_neutrals,
    'Home Points': home_pts_list,
    'Box Score Link': box_score_links,
    'Arena': arenas
}
df_games = pd.DataFrame(data)

# Create a DataFrame for the scraped stats data
columns = ['Team', 'Player', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-']
df_stats = pd.DataFrame(stats_data, columns=columns)

# Print the DataFrames
print("Games DataFrame:")
#print(df_games)
print("\nStats DataFrame:")
#print(df_stats)

Games DataFrame:
                  Date     Visitor/Neutral Visitor Points  \
0    Tue, Oct 18, 2022  Philadelphia 76ers            117   
1    Tue, Oct 18, 2022  Los Angeles Lakers            109   
2    Wed, Oct 19, 2022       Orlando Magic            109   
3    Wed, Oct 19, 2022  Washington Wizards            114   
4    Wed, Oct 19, 2022     Houston Rockets            107   
..                 ...                 ...            ...   
97   Mon, Oct 31, 2022      Indiana Pacers            109   
98   Mon, Oct 31, 2022       Atlanta Hawks            109   
99   Mon, Oct 31, 2022     Detroit Pistons            108   
100  Mon, Oct 31, 2022   Memphis Grizzlies            105   
101  Mon, Oct 31, 2022     Houston Rockets             93   

              Home/Neutral Home Points  \
0           Boston Celtics         126   
1    Golden State Warriors         123   
2          Detroit Pistons         113   
3           Indiana Pacers         107   
4            Atlanta Hawks         117  

In [38]:
df_games.head()

,Date,Visitor/Neutral,Visitor Points,Home/Neutral,Home Points,Box Score Link,Arena
0,"Tue, Oct 18, 2022",Philadelphia 76ers,117,Boston Celtics,126,https://www.basketball-reference.com/boxscores...,TD Garden
1,"Tue, Oct 18, 2022",Los Angeles Lakers,109,Golden State Warriors,123,https://www.basketball-reference.com/boxscores...,Chase Center
2,"Wed, Oct 19, 2022",Orlando Magic,109,Detroit Pistons,113,https://www.basketball-reference.com/boxscores...,Little Caesars Arena
3,"Wed, Oct 19, 2022",Washington Wizards,114,Indiana Pacers,107,https://www.basketball-reference.com/boxscores...,Gainbridge Fieldhouse
4,"Wed, Oct 19, 2022",Houston Rockets,107,Atlanta Hawks,117,https://www.basketball-reference.com/boxscores...,State Farm Arena


In [39]:
df_stats.head()

,Team,Player,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
0,PHI,Tyrese Maxey,38:11,8,16,.500,2,5,.400,3,...,0,1,1,2,2,0,1,5,21,-6
1,PHI,James Harden,37:17,9,14,.643,5,9,.556,12,...,0,8,8,7,0,0,3,3,35,+1
2,PHI,Joel Embiid,37:16,9,18,.500,1,6,.167,7,...,1,14,15,5,0,1,6,4,26,-13
3,PHI,Tobias Harris,34:14,7,14,.500,3,6,.500,1,...,1,1,2,0,3,0,0,3,18,-1
4,PHI,P.J. Tucker,33:01,3,5,.600,0,2,.000,0,...,2,2,4,0,0,1,2,2,6,-6


In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

def scrape_nba_data(year, month):
    # HTML content
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html'
    response = requests.get(url)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table element
    table = soup.find('table', class_='suppress_glossary sortable stats_table')

    # Find all the rows in the table body
    rows = table.tbody.find_all('tr')

    # Create empty lists to store the extracted data
    dates = []
    visitor_neutrals = []
    visitor_pts_list = []
    home_neutrals = []
    home_pts_list = []
    box_score_links = []
    arenas = []

    # Iterate over each row and extract the desired information
    for row in rows:
        date = row.find('th', {'data-stat': 'date_game'}).text.strip()
        visitor_neutral = row.find('td', {'data-stat': 'visitor_team_name'}).text.strip()
        visitor_pts = row.find('td', {'data-stat': 'visitor_pts'}).text.strip()
        home_neutral = row.find('td', {'data-stat': 'home_team_name'}).text.strip()
        home_pts = row.find('td', {'data-stat': 'home_pts'}).text.strip()
        box_score_link = 'https://www.basketball-reference.com' + row.find('td', {'data-stat': 'box_score_text'}).a['href']
        arena = row.find('td', {'data-stat': 'arena_name'}).text.strip()

        dates.append(date)
        visitor_neutrals.append(visitor_neutral)
        visitor_pts_list.append(visitor_pts)
        home_neutrals.append(home_neutral)
        home_pts_list.append(home_pts)
        box_score_links.append(box_score_link)
        arenas.append(arena)

    # Create empty lists to store the combined stats data
    combined_stats_data = []

    # Scrape data from each box score link
    for link in box_score_links:
        box_score_response = requests.get(link)
        box_score_soup = BeautifulSoup(box_score_response.text, 'html.parser')

        # Find the table element with class "sortable stats_table" and an id containing "game-basic"
        stats_table = box_score_soup.find('table', class_='sortable stats_table', id=lambda x: x and 'game-basic' in x)

        if stats_table:
            # Extract the team abbreviation from the id attribute
            team_abbr = stats_table['id'].split('-')[1].upper()

            # Find all the rows in the table body
            stats_rows = stats_table.tbody.find_all('tr')

            # Extract the desired information from each row
            basic_stats_data = []
            for row in stats_rows:
                player = row.find('th', {'data-stat': 'player'})
                player = player.text.strip() if player else ''

                mp = row.find('td', {'data-stat': 'mp'})
                mp = mp.text.strip() if mp else ''

                fg = row.find('td', {'data-stat': 'fg'})
                fg = fg.text.strip() if fg else ''

                fga = row.find('td', {'data-stat': 'fga'})
                fga = fga.text.strip() if fga else ''

                fg_pct = row.find('td', {'data-stat': 'fg_pct'})
                fg_pct = fg_pct.text.strip() if fg_pct else ''

                fg3 = row.find('td', {'data-stat': 'fg3'})
                fg3 = fg3.text.strip() if fg3 else ''

                fg3a = row.find('td', {'data-stat': 'fg3a'})
                fg3a = fg3a.text.strip() if fg3a else ''

                fg3_pct = row.find('td', {'data-stat': 'fg3_pct'})
                fg3_pct = fg3_pct.text.strip() if fg3_pct else ''

                ft = row.find('td', {'data-stat': 'ft'})
                ft = ft.text.strip() if ft else ''

                fta = row.find('td', {'data-stat': 'fta'})
                fta = fta.text.strip() if fta else ''

                ft_pct = row.find('td', {'data-stat': 'ft_pct'})
                ft_pct = ft_pct.text.strip() if ft_pct else ''

                orb = row.find('td', {'data-stat': 'orb'})
                orb = orb.text.strip() if orb else ''

                drb = row.find('td', {'data-stat': 'drb'})
                drb = drb.text.strip() if drb else ''

                trb = row.find('td', {'data-stat': 'trb'})
                trb = trb.text.strip() if trb else ''

                ast = row.find('td', {'data-stat': 'ast'})
                ast = ast.text.strip() if ast else ''

                stl = row.find('td', {'data-stat': 'stl'})
                stl = stl.text.strip() if stl else ''

                blk = row.find('td', {'data-stat': 'blk'})
                blk = blk.text.strip() if blk else ''

                tov = row.find('td', {'data-stat': 'tov'})
                tov = tov.text.strip() if tov else ''

                pf = row.find('td', {'data-stat': 'pf'})
                pf = pf.text.strip() if pf else ''

                pts = row.find('td', {'data-stat': 'pts'})
                pts = pts.text.strip() if pts else ''

                plus_minus = row.find('td', {'data-stat': 'plus_minus'})
                plus_minus = plus_minus.text.strip() if plus_minus else ''

                basic_stats_data.append([player, mp, fg, fga, fg_pct, fg3, fg3a, fg3_pct, ft, fta, ft_pct, orb, drb, trb, ast, stl, blk, tov, pf, pts, plus_minus])

        else:
            print(f"No basic stats table found for {link}")
            continue

        # Find the table element with class "sortable stats_table" and the specific id
        adv_stats_table = box_score_soup.find('table', class_='sortable stats_table', id=f"box-{team_abbr}-game-advanced")

        if adv_stats_table:
            # Find all the rows in the table body
            adv_stats_rows = adv_stats_table.tbody.find_all('tr')

            # Extract the desired information from each row
            adv_stats_data = []
            for row in adv_stats_rows:
                player = row.find('th', {'data-stat': 'player'})
                player = player.text.strip() if player else ''

                mp = row.find('td', {'data-stat': 'mp'})
                mp = mp.text.strip() if mp else ''

                ts_pct = row.find('td', {'data-stat': 'ts_pct'})
                ts_pct = ts_pct.text.strip() if ts_pct else ''

                efg_pct = row.find('td', {'data-stat': 'efg_pct'})
                efg_pct = efg_pct.text.strip() if efg_pct else ''

                fg3a_per_fga_pct = row.find('td', {'data-stat': 'fg3a_per_fga_pct'})
                fg3a_per_fga_pct = fg3a_per_fga_pct.text.strip() if fg3a_per_fga_pct else ''

                fta_per_fga_pct = row.find('td', {'data-stat': 'fta_per_fga_pct'})
                fta_per_fga_pct = fta_per_fga_pct.text.strip() if fta_per_fga_pct else ''

                orb_pct = row.find('td', {'data-stat': 'orb_pct'})
                orb_pct = orb_pct.text.strip() if orb_pct else ''

                drb_pct = row.find('td', {'data-stat': 'drb_pct'})
                drb_pct = drb_pct.text.strip() if drb_pct else ''

                trb_pct = row.find('td', {'data-stat': 'trb_pct'})
                trb_pct = trb_pct.text.strip() if trb_pct else ''

                ast_pct = row.find('td', {'data-stat': 'ast_pct'})
                ast_pct = ast_pct.text.strip() if ast_pct else ''

                stl_pct = row.find('td', {'data-stat': 'stl_pct'})
                stl_pct = stl_pct.text.strip() if stl_pct else ''

                blk_pct = row.find('td', {'data-stat': 'blk_pct'})
                blk_pct = blk_pct.text.strip() if blk_pct else ''

                tov_pct = row.find('td', {'data-stat': 'tov_pct'})
                tov_pct = tov_pct.text.strip() if tov_pct else ''

                usg_pct = row.find('td', {'data-stat': 'usg_pct'})
                usg_pct = usg_pct.text.strip() if usg_pct else ''

                off_rtg = row.find('td', {'data-stat': 'off_rtg'})
                off_rtg = off_rtg.text.strip() if off_rtg else ''

                def_rtg = row.find('td', {'data-stat': 'def_rtg'})
                def_rtg = def_rtg.text.strip() if def_rtg else ''

                bpm = row.find('td', {'data-stat': 'bpm'})
                bpm = bpm.text.strip() if bpm else ''

                adv_stats_data.append([player, mp, ts_pct, efg_pct, fg3a_per_fga_pct, fta_per_fga_pct, orb_pct, drb_pct, trb_pct, ast_pct, stl_pct, blk_pct, tov_pct, usg_pct, off_rtg, def_rtg, bpm])

        else:
            print(f"No advanced stats table found with id 'box-{team_abbr}-game-advanced'")
            continue

        # Combine the basic and advanced stats data based on matching player names
        for basic_stats in basic_stats_data:
            player = basic_stats[0]
            for adv_stats in adv_stats_data:
                if adv_stats[0] == player:
                    combined_stats = [team_abbr] + basic_stats + adv_stats[1:]
                    combined_stats_data.append(combined_stats)
                    break

        # Create a DataFrame for the combined stats data
        columns = ['Team', 'Player', 'MP_basic', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-', 'MP_adv', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
        df_stats = pd.DataFrame(combined_stats_data, columns=columns)

        # Create the directory structure if it doesn't exist
        directory = f"{year}/{month}/{team_abbr}"
        os.makedirs(directory, exist_ok=True)

        # Save the DataFrame to a CSV file in the respective directory
        df_stats.to_csv(f"{directory}/stats.csv", index=False)

        print(f"DataFrame saved to {directory}")

    # Create a DataFrame using the extracted data
    data = {
        'Date': dates,
        'Visitor/Neutral': visitor_neutrals,
        'Visitor Points': visitor_pts_list,
        'Home/Neutral': home_neutrals,
        'Home Points': home_pts_list,
        'Box Score Link': box_score_links,
        'Arena': arenas
    }
    df_games = pd.DataFrame(data)

    # Create the directory structure if it doesn't exist
    directory = f"{year}/{month}"
    os.makedirs(directory, exist_ok=True)

    # Save the DataFrame to a CSV file in the respective directory
    df_games.to_csv(f"{directory}/games.csv", index=False)

    print(f"DataFrame saved to {directory}")




In [57]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

def scrape_nba_data(year, month):
    # HTML content
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html'
    response = requests.get(url)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table element
    table = soup.find('table', class_='suppress_glossary sortable stats_table')

    # Find all the rows in the table body
    rows = table.tbody.find_all('tr')

    # Create empty lists to store the extracted data
    dates = []
    visitor_neutrals = []
    visitor_pts_list = []
    home_neutrals = []
    home_pts_list = []
    box_score_links = []
    arenas = []

    # Iterate over each row and extract the desired information
    for row in rows:
        date = row.find('th', {'data-stat': 'date_game'}).text.strip()
        visitor_neutral = row.find('td', {'data-stat': 'visitor_team_name'}).text.strip()
        visitor_pts = row.find('td', {'data-stat': 'visitor_pts'}).text.strip()
        home_neutral = row.find('td', {'data-stat': 'home_team_name'}).text.strip()
        home_pts = row.find('td', {'data-stat': 'home_pts'}).text.strip()
        box_score_link = 'https://www.basketball-reference.com' + row.find('td', {'data-stat': 'box_score_text'}).a['href']
        arena = row.find('td', {'data-stat': 'arena_name'}).text.strip()

        dates.append(date)
        visitor_neutrals.append(visitor_neutral)
        visitor_pts_list.append(visitor_pts)
        home_neutrals.append(home_neutral)
        home_pts_list.append(home_pts)
        box_score_links.append(box_score_link)
        arenas.append(arena)

    # Create empty lists to store the combined stats data
    combined_stats_data = []

    # Scrape data from each box score link
    for link in box_score_links:
        box_score_response = requests.get(link)
        box_score_soup = BeautifulSoup(box_score_response.text, 'html.parser')

        # Find the table element with class "sortable stats_table" and an id containing "game-basic"
        stats_table = box_score_soup.find('table', class_='sortable stats_table', id=lambda x: x and 'game-basic' in x)

        if stats_table:
            # Extract the team abbreviation from the id attribute
            team_abbr = stats_table['id'].split('-')[1].upper()

            # Find all the rows in the table body
            stats_rows = stats_table.tbody.find_all('tr')

            # Extract the desired information from each row
            basic_stats_data = []
            for row in stats_rows:
                player = row.find('th', {'data-stat': 'player'})
                player = player.text.strip() if player else ''

                mp = row.find('td', {'data-stat': 'mp'})
                mp = mp.text.strip() if mp else ''

                fg = row.find('td', {'data-stat': 'fg'})
                fg = fg.text.strip() if fg else ''

                fga = row.find('td', {'data-stat': 'fga'})
                fga = fga.text.strip() if fga else ''

                fg_pct = row.find('td', {'data-stat': 'fg_pct'})
                fg_pct = fg_pct.text.strip() if fg_pct else ''

                fg3 = row.find('td', {'data-stat': 'fg3'})
                fg3 = fg3.text.strip() if fg3 else ''

                fg3a = row.find('td', {'data-stat': 'fg3a'})
                fg3a = fg3a.text.strip() if fg3a else ''

                fg3_pct = row.find('td', {'data-stat': 'fg3_pct'})
                fg3_pct = fg3_pct.text.strip() if fg3_pct else ''

                ft = row.find('td', {'data-stat': 'ft'})
                ft = ft.text.strip() if ft else ''

                fta = row.find('td', {'data-stat': 'fta'})
                fta = fta.text.strip() if fta else ''

                ft_pct = row.find('td', {'data-stat': 'ft_pct'})
                ft_pct = ft_pct.text.strip() if ft_pct else ''

                orb = row.find('td', {'data-stat': 'orb'})
                orb = orb.text.strip() if orb else ''

                drb = row.find('td', {'data-stat': 'drb'})
                drb = drb.text.strip() if drb else ''

                trb = row.find('td', {'data-stat': 'trb'})
                trb = trb.text.strip() if trb else ''

                ast = row.find('td', {'data-stat': 'ast'})
                ast = ast.text.strip() if ast else ''

                stl = row.find('td', {'data-stat': 'stl'})
                stl = stl.text.strip() if stl else ''

                blk = row.find('td', {'data-stat': 'blk'})
                blk = blk.text.strip() if blk else ''

                tov = row.find('td', {'data-stat': 'tov'})
                tov = tov.text.strip() if tov else ''

                pf = row.find('td', {'data-stat': 'pf'})
                pf = pf.text.strip() if pf else ''

                pts = row.find('td', {'data-stat': 'pts'})
                pts = pts.text.strip() if pts else ''

                plus_minus = row.find('td', {'data-stat': 'plus_minus'})
                plus_minus = plus_minus.text.strip() if plus_minus else ''

                basic_stats_data.append([player, mp, fg, fga, fg_pct, fg3, fg3a, fg3_pct, ft, fta, ft_pct, orb, drb, trb, ast, stl, blk, tov, pf, pts, plus_minus])

        else:
            print(f"No basic stats table found for {link}")
            continue

        # Find the table element with class "sortable stats_table" and the specific id
        adv_stats_table = box_score_soup.find('table', class_='sortable stats_table', id=f"box-{team_abbr}-game-advanced")

        if adv_stats_table:
            # Find all the rows in the table body
            adv_stats_rows = adv_stats_table.tbody.find_all('tr')

            # Extract the desired information from each row
            adv_stats_data = []
            for row in adv_stats_rows:
                player = row.find('th', {'data-stat': 'player'})
                player = player.text.strip() if player else ''

                mp = row.find('td', {'data-stat': 'mp'})
                mp = mp.text.strip() if mp else ''

                ts_pct = row.find('td', {'data-stat': 'ts_pct'})
                ts_pct = ts_pct.text.strip() if ts_pct else ''

                efg_pct = row.find('td', {'data-stat': 'efg_pct'})
                efg_pct = efg_pct.text.strip() if efg_pct else ''

                fg3a_per_fga_pct = row.find('td', {'data-stat': 'fg3a_per_fga_pct'})
                fg3a_per_fga_pct = fg3a_per_fga_pct.text.strip() if fg3a_per_fga_pct else ''

                fta_per_fga_pct = row.find('td', {'data-stat': 'fta_per_fga_pct'})
                fta_per_fga_pct = fta_per_fga_pct.text.strip() if fta_per_fga_pct else ''

                orb_pct = row.find('td', {'data-stat': 'orb_pct'})
                orb_pct = orb_pct.text.strip() if orb_pct else ''

                drb_pct = row.find('td', {'data-stat': 'drb_pct'})
                drb_pct = drb_pct.text.strip() if drb_pct else ''

                trb_pct = row.find('td', {'data-stat': 'trb_pct'})
                trb_pct = trb_pct.text.strip() if trb_pct else ''

                ast_pct = row.find('td', {'data-stat': 'ast_pct'})
                ast_pct = ast_pct.text.strip() if ast_pct else ''

                stl_pct = row.find('td', {'data-stat': 'stl_pct'})
                stl_pct = stl_pct.text.strip() if stl_pct else ''

                blk_pct = row.find('td', {'data-stat': 'blk_pct'})
                blk_pct = blk_pct.text.strip() if blk_pct else ''

                tov_pct = row.find('td', {'data-stat': 'tov_pct'})
                tov_pct = tov_pct.text.strip() if tov_pct else ''

                usg_pct = row.find('td', {'data-stat': 'usg_pct'})
                usg_pct = usg_pct.text.strip() if usg_pct else ''

                off_rtg = row.find('td', {'data-stat': 'off_rtg'})
                off_rtg = off_rtg.text.strip() if off_rtg else ''

                def_rtg = row.find('td', {'data-stat': 'def_rtg'})
                def_rtg = def_rtg.text.strip() if def_rtg else ''

                bpm = row.find('td', {'data-stat': 'bpm'})
                bpm = bpm.text.strip() if bpm else ''

                adv_stats_data.append([player, mp, ts_pct, efg_pct, fg3a_per_fga_pct, fta_per_fga_pct, orb_pct, drb_pct, trb_pct, ast_pct, stl_pct, blk_pct, tov_pct, usg_pct, off_rtg, def_rtg, bpm])

        else:
            print(f"No advanced stats table found with id 'box-{team_abbr}-game-advanced'")
            continue

        # Combine the basic and advanced stats data based on matching player names
        for basic_stats in basic_stats_data:
            player = basic_stats[0]
            for adv_stats in adv_stats_data:
                if adv_stats[0] == player:
                    combined_stats = [team_abbr] + basic_stats + adv_stats[1:]
                    combined_stats_data.append(combined_stats)
                    break

        # Create a DataFrame for the combined stats data
        columns = ['Team', 'Player', 'MP_basic', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-', 'MP_adv', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
        df_stats = pd.DataFrame(combined_stats_data, columns=columns)

        # Create the directory structure if it doesn't exist
        directory = f"{year}/{month}/{team_abbr}"
        os.makedirs(directory, exist_ok=True)

        # Save the DataFrame to a CSV file in the respective directory
        df_stats.to_csv(f"{directory}/stats.csv", index=False)

        print(f"DataFrame saved to {directory}")

        # Filter the games data based on the team abbreviation
        team_games = []
        for i in range(len(dates)):
            if visitor_neutrals[i] == team_abbr or home_neutrals[i] == team_abbr:
                team_games.append([dates[i], visitor_neutrals[i], visitor_pts_list[i], home_neutrals[i], home_pts_list[i], box_score_links[i], arenas[i]])

        # Create a DataFrame for the team's games data
        columns = ['Date', 'Visitor/Neutral', 'Visitor Points', 'Home/Neutral', 'Home Points', 'Box Score Link', 'Arena']
        df_team_games = pd.DataFrame(team_games, columns=columns)

        # Add a game counter column
        df_team_games['Game'] = range(1, len(df_team_games) + 1)

        # Save the team's games DataFrame to a CSV file in the respective directory
        df_team_games.to_csv(f"{directory}/games.csv", index=False)

        print(f"Team {team_abbr} games saved to {directory}")

# Example usage
year = 2023
month = 'october'

scrape_nba_data(year, month)

DataFrame saved to 2023/october/BOS
Team BOS games saved to 2023/october/BOS


In [ ]:
# Example usage
#start_year = 2018
#end_year = 2023
months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']

for year in range(start_year, end_year + 1):
    for month in months:
        try:
            scrape_nba_data(year, month)
        except Exception as e:
            print(f"Error scraping data for {year}/{month}: {str(e)}")

In [52]:
df

,Team,Player,MP,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,PHI,Tyrese Maxey,38:11,.606,.563,.313,.188,0.0,3.8,1.9,8.4,2.6,0.0,5.5,21.7,118,129,-0.9
1,PHI,James Harden,37:17,.908,.821,.643,.857,0.0,31.2,15.4,31.7,0.0,0.0,13.5,27.0,165,132,14.5
2,PHI,Joel Embiid,37:16,.592,.528,.333,.500,3.8,54.6,28.8,22.7,0.0,2.7,21.5,33.9,103,127,-0.2
3,PHI,Tobias Harris,34:14,.605,.607,.429,.143,4.1,4.2,4.2,0.0,4.3,0.0,0.0,19.6,122,125,2.7
4,PHI,P.J. Tucker,33:01,.600,.600,.400,.000,8.6,8.8,8.7,0.0,0.0,3.1,28.6,9.6,98,132,-4.2
5,PHI,Reserves,,,,,,,,,,,,,,,,
6,PHI,De'Anthony Melton,20:33,.625,.625,.500,.000,0.0,0.0,0.0,0.0,2.4,0.0,0.0,8.8,122,130,-2.0
7,PHI,Danuel House Jr.,16:12,.174,.000,.500,1.000,0.0,9.0,4.4,7.4,6.0,0.0,25.8,10.8,43,121,-5.6
8,PHI,Georges Niang,12:09,.375,.375,.750,.000,0.0,0.0,0.0,11.0,0.0,0.0,0.0,14.9,88,136,-6.6
9,PHI,Montrezl Harrell,10:44,.333,.333,.000,.000,0.0,0.0,0.0,0.0,0.0,9.5,25.0,16.8,49,128,-16.3


In [ ]:
## df1 = get regular season data at team website with game #
## df2 = get individual game stats and advanced stats and concat - function incorproates season, team 
    ## save date for each game
## save to directory by year/team

## Scrape Season data by year/team

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

def scrape_data_season(team, year):
    # HTML content
    url = f'https://www.basketball-reference.com/teams/{team}/{year}_games.html'
    response = requests.get(url)
    
    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the table element
    table = soup.find('table', class_='sortable stats_table')
    
    # Find all the rows in the table body
    rows = table.tbody.find_all('tr')
    
    # Create empty lists to store the extracted data
    game_number = []
    dates = []
    start_times = []
    networks = []
    visitor_neutrals = []
    visitor_pts_list = []
    home_neutrals = []
    home_pts_list = []
    box_score_links = []
    arenas = []
    game_results = []
    overtimes = []
    team_pts = []
    opp_pts = []
    wins = []
    losses = []
    game_streaks = []
    notes = []
    
    # Iterate over each row and extract the desired information
    for row in rows:
        game_num = row.find('th', {'data-stat': 'g'})
        date = row.find('td', {'data-stat': 'date_game'})
        start_time = row.find('td', {'data-stat': 'game_start_time'})
        network = row.find('td', {'data-stat': 'network'})
        visitor_neutral = row.find('td', {'data-stat': 'opp_name'})
        visitor_pts = row.find('td', {'data-stat': 'opp_pts'})
        home_neutral = team
        home_pts = row.find('td', {'data-stat': 'pts'})
        box_score_link = row.find('td', {'data-stat': 'box_score_text'})
        arena = row.find('td', {'data-stat': 'game_location'})
        game_result = row.find('td', {'data-stat': 'game_result'})
        overtime = row.find('td', {'data-stat': 'overtimes'})
        team_pt = row.find('td', {'data-stat': 'pts'})
        opp_pt = row.find('td', {'data-stat': 'opp_pts'})
        win = row.find('td', {'data-stat': 'wins'})
        loss = row.find('td', {'data-stat': 'losses'})
        game_streak = row.find('td', {'data-stat': 'game_streak'})
        note = row.find('td', {'data-stat': 'game_remarks'})
        
        game_number.append(game_num.text.strip() if game_num else '')
        dates.append(date.text.strip() if date else '')
        start_times.append(start_time.text.strip() if start_time else '')
        networks.append(network.text.strip() if network else '')
        visitor_neutrals.append(visitor_neutral.text.strip() if visitor_neutral else '')
        visitor_pts_list.append(visitor_pts.text.strip() if visitor_pts else '')
        home_neutrals.append(home_neutral)
        home_pts_list.append(home_pts.text.strip() if home_pts else '')
        box_score_links.append('https://www.basketball-reference.com' + box_score_link.a['href'] if box_score_link and box_score_link.a else '')
        arenas.append('Home' if arena and arena.text.strip() == '' else 'Away')
        game_results.append(game_result.text.strip() if game_result else '')
        overtimes.append(overtime.text.strip() if overtime else '')
        team_pts.append(team_pt.text.strip() if team_pt else '')
        opp_pts.append(opp_pt.text.strip() if opp_pt else '')
        wins.append(win.text.strip() if win else '')
        losses.append(loss.text.strip() if loss else '')
        game_streaks.append(game_streak.text.strip() if game_streak else '')
        notes.append(note.text.strip() if note else '')
    
    # Create a DataFrame for the team's games data
    columns = ['Game', 'Date', 'Start Time', 'Network', 'Visitor/Neutral', 'Visitor Points', 'Home/Neutral', 'Home Points',
               'Box Score Link', 'Arena', 'Game Result', 'Overtime', 'Team Points', 'Opponent Points', 'Wins', 'Losses',
               'Game Streak', 'Notes']
    df_team_games = pd.DataFrame({'Game': game_number, 'Date': dates, 'Start Time': start_times, 'Network': networks,
                                  'Visitor/Neutral': visitor_neutrals, 'Visitor Points': visitor_pts_list,
                                  'Home/Neutral': home_neutrals, 'Home Points': home_pts_list,
                                  'Box Score Link': box_score_links, 'Arena': arenas, 'Game Result': game_results,
                                  'Overtime': overtimes, 'Team Points': team_pts, 'Opponent Points': opp_pts,
                                  'Wins': wins, 'Losses': losses, 'Game Streak': game_streaks, 'Notes': notes},
                                 columns=columns)
    
    # Remove rows where the value in the 'G' column is "G"
    df_team_games = df_team_games[df_team_games['Game'] != 'G']
    
    # Create the directory if it doesn't exist
    directory = f"{year}/{team}"
    os.makedirs(directory, exist_ok=True)
    
    # Save the DataFrame as a CSV file in the specified directory
    csv_file_path = f"{directory}/{team}_{year}_games.csv"
    df_team_games.to_csv(csv_file_path, index=False)
    
    print(f"CSV file saved: {csv_file_path}")
    
    return df_team_games



In [23]:
# NBA team abbreviations
nba_teams = ['ATL', 'BOS', 'BRK', 'CHI', 'CHO', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

# Iterate over the years and teams
for year in range(2018, 2025):
    for team in nba_teams:
        scrape_data_season(team, year)

CSV file saved: 2018/ATL/ATL_2018_games.csv
CSV file saved: 2018/BOS/BOS_2018_games.csv
CSV file saved: 2018/BRK/BRK_2018_games.csv
CSV file saved: 2018/CHI/CHI_2018_games.csv
CSV file saved: 2018/CHO/CHO_2018_games.csv
CSV file saved: 2018/CLE/CLE_2018_games.csv
CSV file saved: 2018/DAL/DAL_2018_games.csv
CSV file saved: 2018/DEN/DEN_2018_games.csv
CSV file saved: 2018/DET/DET_2018_games.csv
CSV file saved: 2018/GSW/GSW_2018_games.csv
CSV file saved: 2018/HOU/HOU_2018_games.csv
CSV file saved: 2018/IND/IND_2018_games.csv
CSV file saved: 2018/LAC/LAC_2018_games.csv
CSV file saved: 2018/LAL/LAL_2018_games.csv
CSV file saved: 2018/MEM/MEM_2018_games.csv
CSV file saved: 2018/MIA/MIA_2018_games.csv
CSV file saved: 2018/MIL/MIL_2018_games.csv
CSV file saved: 2018/MIN/MIN_2018_games.csv
CSV file saved: 2018/NOP/NOP_2018_games.csv
CSV file saved: 2018/NYK/NYK_2018_games.csv
CSV file saved: 2018/OKC/OKC_2018_games.csv
CSV file saved: 2018/ORL/ORL_2018_games.csv
CSV file saved: 2018/PHI/PHI_201